In [1]:
# libraries
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
 
# Build a dataframe with your connections
df = pd.DataFrame({ 'from':['A', 'B', 'C','A','E','F','E','G','G','D','F'], 'to':['D', 'A', 'E','C','A','F','G','D','B','G','C']})
df
 

,from,to
0,A,D
1,B,A
2,C,E
3,A,C
4,E,A
5,F,F
6,E,G
7,G,D
8,G,B
9,D,G


Networkx plot function and automatic layouts
https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout

In [ ]:
# Build your graph
G = nx.from_pandas_edgelist(df, 'from', 'to')

S=[1000,1000,2000,2000,3000,4000,3000]

# Fruchterman Reingold
nx.draw(G, with_labels=True, node_size=S, node_color="skyblue", pos=nx.fruchterman_reingold_layout(G))
plt.title("fruchterman_reingold")
plt.show()
 
# Circular
nx.draw(G, with_labels=True, node_size=1500, node_color="skyblue", pos=nx.circular_layout(G))
plt.title("circular")
plt.show()
 
# Random
nx.draw(G, with_labels=True, node_size=1500, node_color="skyblue", pos=nx.random_layout(G))
plt.title("random")
plt.show()
 
# Spectral
nx.draw(G, with_labels=True, node_size=1500, node_color="skyblue", pos=nx.spectral_layout(G))
plt.title("spectral")
plt.show()
 
# Spring
nx.draw(G, with_labels=True, node_size=2000, node_color="skyblue", pos=nx.spring_layout(G))
plt.title("spring")
plt.show()



Organize tree visualization


In [ ]:
# Build a dataframe with your connections
df = pd.DataFrame({ 'from':['A', 'A', 'A','B','B','C','C'], 'to':['B', 'C', 'D', 'E','F','G','H']})
G=nx.from_pandas_edgelist(df, 'from', 'to')
df

In [ ]:
def hierarchy_pos(G, root=None, width=1., vert_gap = 0.2, vert_loc = 0, xcenter = 0.5):

    '''
    From Joel's answer at https://stackoverflow.com/a/29597209/2966723.  
    Licensed under Creative Commons Attribution-Share Alike 
    
    If the graph is a tree this will return the positions to plot this in a 
    hierarchical layout.
    
    G: the graph (must be a tree)
    
    root: the root node of current branch 
    - if the tree is directed and this is not given, 
      the root will be found and used
    - if the tree is directed and this is given, then 
      the positions will be just for the descendants of this node.
    - if the tree is undirected and not given, 
      then a random choice will be used.
    
    width: horizontal space allocated for this branch - avoids overlap with other branches
    
    vert_gap: gap between levels of hierarchy
    
    vert_loc: vertical location of root
    
    xcenter: horizontal location of root
    '''
    if not nx.is_tree(G):
        raise TypeError('cannot use hierarchy_pos on a graph that is not a tree')

    if root is None:
        if isinstance(G, nx.DiGraph):
            root = next(iter(nx.topological_sort(G)))  #allows back compatibility with nx version 1.11
        else:
            root = random.choice(list(G.nodes))

    def _hierarchy_pos(G, root, width=1., vert_gap = 0.2, vert_loc = 0, xcenter = 0.5, pos = None, parent = None):
        '''
        see hierarchy_pos docstring for most arguments

        pos: a dict saying where all nodes go if they have been assigned
        parent: parent of this branch. - only affects it if non-directed

        '''
    
        if pos is None:
            pos = {root:(xcenter,vert_loc)}
        else:
            pos[root] = (xcenter, vert_loc)
        children = list(G.neighbors(root))
        if not isinstance(G, nx.DiGraph) and parent is not None:
            children.remove(parent)  
        if len(children)!=0:
            dx = width/len(children) 
            nextx = xcenter - width/2 - dx/2
            for child in children:
                nextx += dx
                pos = _hierarchy_pos(G,child, width = dx, vert_gap = vert_gap, 
                                    vert_loc = vert_loc-vert_gap, xcenter=nextx,
                                    pos=pos, parent = root)
        return pos

            
    return _hierarchy_pos(G, root, width, vert_gap, vert_loc, xcenter)

values=[1000, 1400, 1200, 1000, 200, 600, 600, 400]
# Random
pos = hierarchy_pos(G,'A')   
nx.draw(G, with_labels=True, node_size=values, node_color="skyblue", pos=pos)


plt.title("tree")
plt.show()

<h3> Treemap in plotly

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px

In [ ]:


fig = px.treemap(
    names = ["A", "B", "C","D","E","F","G","H"],
    parents=["", "A", "A", "A", "B", "B", "C", "C" ],
    values=[10, 14, 12, 10, 2, 6, 6, 4]
)
  
fig.show()

Sunburst in plotly

In [ ]:
fig =px.sunburst(
    names = ["A", "B", "C","D","E","F","G","H"],
    parents=["", "", "A", "A", "B", "B", "C", "C" ],
    values=[10, 14, 12, 10, 2, 6, 6, 4]
)
fig.show()

Exercise: think about different task on tree visualizations and which could be the best solution given the previous examples
    

<h3> Interactive network </h3>
We can use pyvis to display interactive node-link diagrams

In [ ]:
from pyvis.network import Network 
import networkx as nx

In [ ]:
net = Network(notebook=True)
net.add_nodes([1, 2, 3, 4], 
label=['A', 'B', 'C', 'D'],
color=['#3da831', '#9a31a8', '#3155a8', '#eb4034'])
net.show('list_of_nodes_with_color.html')

In [ ]:
net.add_edges([(1, 2), (3, 4), (1, 3)])
net.show('edges.html')

In [ ]:
net.add_edge(1, 4, value=2)
net.show('edges_with_weights.html')

In [ ]:
#change default parameters
net.repulsion(node_distance=30, spring_length=100)
net.show('edges_with_weight.html')
#net.save_graph('edges_with_weight.html')

we can add GUI elements!

In [ ]:
net.show_buttons(filter_=True)
net.show('edges_with_weights.html')

directed graphs:

In [ ]:
net = Network(
    notebook=True,
    directed=True
)
net.add_nodes([1, 2, 3, 4], 
label=['A', 'B', 'C', 'D'],
color=['#3da831', '#9a31a8', '#3155a8', '#eb4034'])
net.show('list_of_nodes_with_color.html')
net.add_edges([(1, 2), (3, 4), (1, 3)])

net.show('edges_with_weights.html')

we can import networks from the Networkx encoding

In [ ]:
g = Network(notebook=True)
nxg=nx.complete_graph(5)
g.from_nx(nxg)
g.show("network.html")

Building node-link diagram with plotly (scatter plot)

In [ ]:
import plotly.graph_objects as go
G = nx.random_geometric_graph(200, 0.125)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

In [ ]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

Exercise: using data of previous  lectures, create a treemap visualization for GDP or population or covid cases in different continent->countries mapping values with color
Look at plotly documentation/examples
